In [3]:
!pip install faiss-cpu sentence-transformers transformers
!pip install pinecone
!pip install cohere
!pip install --upgrade pinecone-client sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 63.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.8/419.8 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.1/233.1 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 82.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 91.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/2

In [4]:
import pinecone
import cohere


In [5]:
import os
from pinecone import Pinecone, ServerlessSpec
from sentence_transformers import SentenceTransformer

# Initialize Pinecone with your API key
pc = Pinecone(api_key="7de99cc8-1389-4912-9a1b-1f3e3e7ca42c")

# Set the name for the index we'll be using
index_name = "test1"

# Check if the index already exists; if not, create it
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,
        metric='euclidean',
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )

index = pc.Index(index_name)

cohere_client = cohere.Client('SB102vS4TC1TDuUh6br0DTPHU4wzChX4UHREfUar')

# Load the Sentence-BERT model for generating embeddings
embedder = SentenceTransformer('all-MiniLM-L6-v2')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
# Sample documents that we'll store in the vector database
documents = [
    "Pinecone is a vector database for similarity search.",
    "Cohere provides NLP models for text generation and question answering.",
    "RAG combines retrieval and generation techniques for answering queries."
]


In [7]:
# Generate embeddings for the sample documents
doc_embeddings = embedder.encode(documents)
ids = [f'doc_{i}' for i in range(len(documents))]
vectors = list(zip(ids, doc_embeddings))
index.upsert(vectors)


{'upserted_count': 3}

In [8]:
# Fetch the most relevant documents based on the user's query.
def retrieve_relevant_docs(query, top_k=3):
    query_embedding = embedder.encode([query]).tolist()
    results = index.query(vector=query_embedding, top_k=top_k)
    retrieved_docs = [documents[int(res['id'].split('_')[1])] for res in results['matches']]
    return retrieved_docs


In [9]:
# Generate an answer using context from retrieved documents.
def generate_answer(query, context_docs):
    context = " ".join(context_docs)
    prompt = f"Context: {context}\n\nQuestion: {query}\nAnswer:"
    response = cohere_client.generate(
        model='command-xlarge',
        prompt=prompt,
        max_tokens=150
    )
    return response.generations[0].text.strip()


In [10]:
queries = [
    "What is Pinecone?",
    "What does Cohere provide?",
    "What is RAG?"
]

for query in queries:
    retrieved_docs = retrieve_relevant_docs(query)
    answer = generate_answer(query, retrieved_docs)
    print(f"Question: {query}")
    print(f"Answer: {answer}\n")


Question: What is Pinecone?
Answer: Pinecone is a vector database for similarity search.

Question: What does Cohere provide?
Answer: Cohere provides NLP models for text generation and question answering.

Question: What is RAG?
Answer: RAG is a technique that combines retrieval and generation techniques for answering queries.

